This tutorial is adapted from Carson Sievert's [Plotly for R Master Class](https://plotly-book.cpsievert.me/index.html).


# Combining Views

## Setting up

In [1]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

txhousing = pd.read_csv("txhousing.csv")
txhousing.head()

,city,year,month,sales,volume,median,listings,inventory,date
0,Abilene,2000,1,72.0,5380000.0,71400.0,701.0,6.3,2000.000000
1,Abilene,2000,2,98.0,6505000.0,58700.0,746.0,6.6,2000.083333
2,Abilene,2000,3,130.0,9285000.0,58100.0,784.0,6.8,2000.166667
3,Abilene,2000,4,98.0,9730000.0,68600.0,785.0,6.9,2000.250000
4,Abilene,2000,5,141.0,10590000.0,67300.0,794.0,6.8,2000.333333


## With `plotly.express`

Currently, Plotly Express does not support mixed subplots. However, we can make facet plots and marginal distribution plots.

### Facets 

In [2]:
txhousing_2000 = txhousing.loc[((txhousing.city == "Abilene") | (txhousing.city == "Wichita Falls")) & (txhousing.year == 2000)]

fig = px.scatter(txhousing_2000, x = "sales", y = "listings", facet_col = "city")
fig.show()

In [3]:
txhousing_2000 = txhousing.loc[((txhousing.city == "Abilene") | (txhousing.city == "Wichita Falls")) & (txhousing.year == 2000)]

fig = px.scatter(txhousing_2000, x = "sales", y = "listings", facet_row = "city")
fig.show()

In [4]:
fig = px.scatter(txhousing, x = "sales", y = "listings",
                facet_col='city', facet_col_wrap=10)
fig.show()

### Marginal Distribution Plots 

In [5]:
penguins = pd.read_csv("palmerpenguins.csv")

fig = px.density_heatmap(penguins, x = "bill_length_mm", y = "flipper_length_mm", marginal_x = "box", marginal_y = "violin")
fig.show()

## With `plotly.subplots`

We can use the plotly subplots library in conjunction with plotly graph objects to make multiple views as well. 

In [9]:
from plotly.subplots import make_subplots

fig = go.Figure()

fig = make_subplots(rows = 1, cols = 2,
                    specs = [[{"type":"scatter"}, {"type":"scatter"}]])

fig.add_trace(go.Scatter(x = txhousing["date"], y = txhousing["median"],
                        mode = 'lines', line = go.scatter.Line(color = "blue"), showlegend = False),
                        row = 1, col = 2)
fig.add_trace(go.Scatter(x = txhousing["sales"], y = txhousing["listings"],
                        mode = "markers", showlegend = False),
                        row = 1, col = 1)




## With `dash`

Dash is a framework for building analytical apps that is integrated with the Plotly graphing library. Install Dash here: [https://dash.plotly.com/](https://dash.plotly.com/), and find the fundamentals here: [https://dash.plotly.com/layout](https://dash.plotly.com/layout)

In [10]:
from dash import Dash, dcc, html

app = Dash()
app.layout = html.Div([
    dcc.Graph(figure = fig)
])

app.run_server(debug = True, use_reloader = False) # turn off reloader inside jupyter

## Brushing and Linking

Dash also supports custom interactivity through callbacks-- functions that are called when triggered by events suck as clicking or hovering. Reference for brushing and linking can be found here: [https://dash.plotly.com/interactive-graphing#generic-crossfilter-recipe](https://dash.plotly.com/interactive-graphing#generic-crossfilter-recipe) (code below based on this example).  

In [17]:
from dash import Input, Output, callback
import numpy as np 

app = Dash()

# make a sample data frame with 6 columns
np.random.seed(0)  # no-display
df = pd.DataFrame({"Col " + str(i + 1): np.random.rand(30) for i in range(6)})

app.layout = html.Div(
    [
        html.Div(
            dcc.Graph(id="g1", config={"displayModeBar": False}),
            className="four columns",
        ),
        html.Div(
            dcc.Graph(id="g2", config={"displayModeBar": False}),
            className="four columns",
        ),
        html.Div(
            dcc.Graph(id="g3", config={"displayModeBar": False}),
            className="four columns",
        ),
    ],
    className="row",
)


def get_figure(df, x_col, y_col, selectedpoints, selectedpoints_local):

    if selectedpoints_local and selectedpoints_local["range"]:
        ranges = selectedpoints_local["range"]
        selection_bounds = {
            "x0": ranges["x"][0],
            "x1": ranges["x"][1],
            "y0": ranges["y"][0],
            "y1": ranges["y"][1],
        }
    else:
        selection_bounds = {
            "x0": np.min(df[x_col]),
            "x1": np.max(df[x_col]),
            "y0": np.min(df[y_col]),
            "y1": np.max(df[y_col]),
        }

    # set which points are selected with the `selectedpoints` property
    # and style those points with the `selected` and `unselected`
    # attribute. see
    # https://medium.com/@plotlygraphs/notes-from-the-latest-plotly-js-release-b035a5b43e21
    # for an explanation
    fig = px.scatter(df, x=df[x_col], y=df[y_col], text=df.index)

    fig.update_traces(
        selectedpoints=selectedpoints,
        customdata=df.index,
        mode="markers+text",
        marker={"color": "rgba(0, 116, 217, 0.7)", "size": 20},
        unselected={
            "marker": {"opacity": 0.3},
            "textfont": {"color": "rgba(0, 0, 0, 0)"},
        },
    )

    fig.update_layout(
        margin={"l": 20, "r": 0, "b": 15, "t": 5},
        dragmode="select",
        hovermode=False,
        newselection_mode="gradual",
    )

    fig.add_shape(
        dict(
            {"type": "rect", "line": {"width": 1, "dash": "dot", "color": "darkgrey"}},
            **selection_bounds
        )
    )
    return fig


# this callback defines 3 figures
# as a function of the intersection of their 3 selections
@callback(
    Output("g1", "figure"),
    Output("g2", "figure"),
    Output("g3", "figure"),
    Input("g1", "selectedData"),
    Input("g2", "selectedData"),
    Input("g3", "selectedData"),
)
def callback(selection1, selection2, selection3):
    selectedpoints = df.index
    for selected_data in [selection1, selection2, selection3]:
        if selected_data and selected_data["points"]:
            selectedpoints = np.intersect1d(
                selectedpoints, [p["customdata"] for p in selected_data["points"]]
            )

    return [
        get_figure(df, "Col 1", "Col 2", selectedpoints, selection1),
        get_figure(df, "Col 3", "Col 4", selectedpoints, selection2),
        get_figure(df, "Col 5", "Col 6", selectedpoints, selection3),
    ]


app.run(debug=True)





# Your turn! 
Using the `palmerspenguins` dataset create:
1. A scatterplot showing bill_length_mm by flipper_length_mm, colored by species. 
2. A barchart showing counts of penguins across species. 
3. Link the two charts such that brushing over points in the scatterplot causes the barchart to show counts only for selected points.  